# ETUDE DYNAMYQUE DES METROS DE PARIS

## Introduction

Nous allons faire une étude sur le nombre de personne qui attentte a chaque métros.

paramètres:

* nb : nb de personne
* ar : % de personne qui arrive 
* so : % de perssonne qui sorte de la gare à force d'atendre
* ta : temps d'arriver du metro
* cm : capacité du metro



paramètres plus tard
* temps pour une perssonne a rentré / sortir

### Les importation

In [ ]:
%matplotlib inline

from pylab import *

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
#ligne;nom;niem_arret;
ar = 0
so = 0
ta = 2
cm = 0.5
    

# initialisation
on va commencer par deux gares relier entre elles.  
hypothèse :  
 * deplacement wagon instantané

In [ ]:
N = 100
TMAX = 24*3600 + 1 # permet de reinitialiser le temps toute les 24h

class Station:
    def __init__(self, ar, so, name):
        self.ar = ar # % d'arrivant dans la gare
        self.so = so # % de partant dans la gare
        self.nb = N # nombre de personne dans le metro
        self.t = 0   # initialisatien du temps de la gare
        self.ta = [] # temps entre chaque metro
        self.cm = [] # capacité d'un metro
        self.next_s = [] # la prochaine gare
        self.name = name
    
    def update(self): # fonction qui actualise les donnés
        if self.so >= np.random.choice(100):
            n = np.random.choice(10)
            if self.nb - n >= 0:
                self.nb = self.nb - n
            else:
                self.nb = 0
        if self.ar >= np.random.choice(100):
            self.nb += np.random.choice(10)
        self.t = (self.t + 1) % TMAX
        for i in range(len(self.next_s)):
            if self.t % self.ta[i] == 0: #si metro est arrive
                if self.nb >= self.cm[i]: #si plus de monde que de place
                    #regarde si voy est arriver si cest pas le cas tu rajoute dans next
                    self.nb = self.nb - self.cm[i]
                else: #plus de place
                    self.nb = 0
                
    
    def next_station(self, station, ta, cm):
        self.next_s.append(station)
        self.ta.append(ta)
        self.cm.append(cm)

In [ ]:
nb_step = 24*3600 #temps de lanalyse

cm = 700

ar1 = 32
so1 = 5
ta1 = 10*60 + 0 #min * 60 + sec

ar2 = 32
so2 = 5
ta2 = 10*60 + 0 #min * 60 + sec

## une simulation

In [ ]:
TIME_CAPT = 1 + 0 # temps ent chaque prise d'info sur le nombre de perssonne en min*60 + sec (le tous sera en sec) 

def simulation(L):
    time = [] # liste qui represente le temps 
    nb_tot = []
    nb_station = []
    for t in range(nb_step):
        for j in L:
            j.update()
        if t % TIME_CAPT == 0:
            nb = 0
            time.append(t)
            k = 0
            for j in L:
                if k >= len(nb_station):
                    nb_station.append([])
                nb_station[k].append(j.nb)
                nb += j.nb
                k+=1
            nb_tot.append(nb)
    return (time, nb_tot, nb_station)


    
    

In [ ]:
def init(List):
    List = []
    List_Gare.append(Station(ar1, so1,"gare 1"))
    List_Gare.append(Station(ar2, so2, "gare 2"))
    List[0].next_station(List_Gare[1], ta1, cm)
    List[1].next_station(List_Gare[0], ta2, cm)
    return List

In [ ]:
def dessin(List_Gare):
    legende = []
    time, nb_tot, nb_station = simulation(List_Gare)
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14,7))
    axes[0].plot(time, nb_tot, 'b')
    axes[0].set_xlabel("temps")
    axes[0].set_ylabel("nb de personne au total")
    for i in range(len(nb_station)):
        axes[1].plot(time, nb_station[i])
        legende.append('gare ' + str(i + 1))
    axes[1].legend(legende)
    axes[1].set_xlabel("temps")
    axes[1].set_ylabel("nb de personne par gare")

In [ ]:
List_Gare = []
List_Gare.append(Station(ar1, so1,"gare 1"))
List_Gare.append(Station(ar2, so2, "gare 2"))
List_Gare[0].next_station(List_Gare[1], ta1, cm)
List_Gare[1].next_station(List_Gare[0], ta2, cm)


dessin(List_Gare)


## moyenner

In [ ]:
ERR_INTERVAL = 1
def dessin_err(List_Gare, L_err, time, nb_tot): 
    legende = []
    time_err = [time[i] for i in range(len(time)) if i % ERR_INTERVAL  == 0]
    nb_tot_err = [nb_tot[i] for i in range(len(nb_tot)) if i % ERR_INTERVAL == 0]
    L_err_min = [L_err[i][0] for i in range(len(L_err)) if i % ERR_INTERVAL == 0]
    L_err_max = [L_err[i][1] for i in range(len(L_err)) if i % ERR_INTERVAL == 0]
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14,7))
    axes[0].plot(time, nb_tot, 'b')
    axes[0].errorbar(time_err, nb_tot_err, yerr=[L_err_min, L_err_max], ecolor='r')
    axes[0].set_xlabel("temps")
    axes[0].set_ylabel("nb de personne au total")
    for i in range(len(nb_station)):
        axes[1].plot(time, nb_station[i])
        legende.append('gare ' + str(i + 1))
    axes[1].legend(legende)
    axes[1].set_xlabel("temps")
    axes[1].set_ylabel("nb de personne par gare")

In [ ]:
time, nb_tot_moy, nb_station_moy = simulation(List_Gare)

repet = 100

L=[]
L.append(nb_tot_moy.copy())
#initialisation 
for j in range(len(nb_tot_moy)):
    nb_tot_moy[j] = nb_tot_moy[j] / repet

for j in range(len(nb_station_moy)):
    for k in range(len(nb_station_moy[j])):
        nb_station_moy[j][k] = nb_station_moy[j][k] / repet

#recuperation des autre simulation
for i in range(repet - 1):
    #remise a 0
    List_Gare = []
    List_Gare.append(Station(ar1, so1,"gare 1"))
    List_Gare.append(Station(ar2, so2, "gare 2"))
    List_Gare[0].next_station(List_Gare[1], ta1, cm)
    List_Gare[1].next_station(List_Gare[0], ta2, cm)
    time, nb_tot, nb_station = simulation(List_Gare)
    L.append(nb_tot.copy())
    for j in range(len(nb_tot)):
        nb_tot_moy[j] += nb_tot[j] / repet
    for j in range(len(nb_station)):
        for k in range(len(nb_station[j])):
            nb_station[j][k] += nb_station[j][k] / repet

            
L_err = [] # initialisation des barre d'érreur
x=0
for k in range(len(L[0])):
    for i in range(len(L[0])):
        for j in range(len(L)-1):
            if L[j][k]>L[j+1][k]:
                x=L[j][k]
                L[j][k]=L[j+1][k]
                L[j+1][k]=x
    L_err.append([nb_tot_moy[k] - L[round(repet*0.025)][k], L[round(repet*0.975)][k] - nb_tot_moy[k]])

    
dessin_err(List_Gare, L_err, time, nb_tot)

## Cas avec 3 gares où les métros font des allers/retours 

In [ ]:
ar3 = 32
so3 = 5
ta3 =ta2

List_Gare = []

List_Gare.append(Station(ar1, so1))
List_Gare.append(Station(ar2, so2))
List_Gare.append(Station(ar3, so3))

List_Gare[0].next_station(List_Gare[1], ta1, cm)
List_Gare[1].next_station(List_Gare[0], ta2, cm)
List_Gare[1].next_station(List_Gare[2], ta3, cm)
List_Gare[2].next_station(List_Gare[1], ta3, cm)


dessin(List_Gare)

In [ ]:
time, nb_tot_moy, nb_station_moy = simulation(List_Gare)

repet = 100 # nb de repetition pour le moyennage.

L=[]
L.append(nb_tot_moy.copy())
#initialisation 
for j in range(len(nb_tot_moy)):
    nb_tot_moy[j] = nb_tot_moy[j] / repet

for j in range(len(nb_station)):
    for k in range(len(nb_station[j])):
        nb_station[j][k] = nb_station[j][k] / repet

#recuperation des autre simulation
for i in range(repet - 1):
    #remise a 0
    List_Gare = []
    List_Gare.append(Station(ar1, so1,"gare 1"))
    List_Gare.append(Station(ar2, so2, "gare 2"))
    List_Gare.append(Station(ar3, so3, "gare 2"))
    List_Gare[0].next_station(List_Gare[1], ta1, cm)
    List_Gare[1].next_station(List_Gare[0], ta2, cm)
    List_Gare[1].next_station(List_Gare[2], ta2, cm)
    List_Gare[2].next_station(List_Gare[1], ta2, cm)
    
    
    time, nb_tot, nb_station =simulation(List_Gare)
    L.append(nb_tot.copy())
    for j in range(len(nb_tot)):
        nb_tot_moy[j] += nb_tot[j] / repet
    for j in range(len(nb_station)):
        for k in range(len(nb_station[j])):
            nb_station[j][k] += nb_station[j][k] / repet

            
L_err = []
x=0

for i in range(len(L[0])):
    for j in range(len(L)-1):
        if L[j][k]>L[j+1][k]:
            x=L[j][k]
            L[j][k]=L[j+1][k]
            L[j+1][k]=x
                
for k in range(len(L[0])):
    L_err.append([nb_tot_moy[k] - L[round(repet*0.025)][k], L[round(repet*0.975)][k] - nb_tot_moy[k]])

    
dessin_err(List_Gare, L_err, time, nb_tot)